# Exploratory Data Analysis

The data originates from the [kaggle copetition](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview)

<p align="center">
    <img src="img/description.png"  width="80%" height="20%">
</p>

### Dataset Description
File descriptions
 * train.csv - the training set.
 * test.csv - the test set.
 * data_description.txt - full description of each column, originally prepared by Dean De Cock but lightly edited to match the column names used here.

 #### EDA:

 **Target Variable**
 
The variable we aim to predict is `SalePrice`.

In [ ]:
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from scipy import stats
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from scipy.stats import shapiro  
from statsmodels.stats.outliers_influence import variance_inflation_factor
from plot_tools import plot_distribution, plot_corration_map, PlotRelations
from bayesian_opt import Optimizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from skopt.space import Integer, Real, Categorical

In [ ]:
df = pd.read_csv('data/train.csv')
display(df.head())

In [ ]:
df_target = df[['SalePrice']]
df_features = df.drop(['Id', 'SalePrice'], axis=1)

### Feature engineering

First, let's examine the missing data and input it accordingly.

In [ ]:
def get_missing(X: pd.DataFrame) -> pd.DataFrame:
    missing_rate = (X.isnull().sum() / len(X)) * 100
    missing_rate = missing_rate.drop(
        missing_rate[missing_rate == 0].index).sort_values(ascending=False)[:30]
    missing_data = pd.DataFrame({'Missing Ratio': missing_rate})
    return missing_data

In [ ]:
get_missing(df_target)

In [ ]:
missing_data = get_missing(df_features)
display(missing_data.head(10))

Alley: Type of alley access to property

* NA: 	No alley access
		
BsmtQual: Evaluates the height of the basement

* NA:	No Basement
		
BsmtCond: Evaluates the general condition of the basement

* NA:	No Basement
	
BsmtExposure: Refers to walkout or garden level walls

* NA:	No Basement
	
BsmtFinType1: Rating of basement finished area

* NA:	No Basement
		
BsmtFinType2: Rating of basement finished area (if multiple types)

* NA:	No Basement

FireplaceQu: Fireplace quality
* NA:	No Fireplace
		
GarageType: Garage location
* NA:	No Garage
	
GarageFinish: Interior finish of the garage
* NA:	No Garage

GarageQual: Garage quality
* NA:	No Garage
		
GarageCond: Garage condition
* NA:	No Garage

PoolQC: Pool quality
* NA:	No Pool
		
Fence: Fence quality
* NA:	No Fence
	
MiscFeature: Miscellaneous feature not covered in other categories
* NA:	None



In [ ]:
variables_where_null_is_0 = [
                'BsmtQual', 'BsmtCond', 'BsmtExposure', 
                'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 
                'GarageType', 'GarageFinish', 'GarageQual', 
                'GarageCond', 'PoolQC', 'Fence'
            ]
values = {"Functional": "Typ", 
          "Alley": "None", 
          "MasVnrType": "None", 
          "MiscFeature": "no_misc_feature", 
          **{v:0 for v in variables_where_null_is_0}}
df_features.fillna(value=values, inplace=True)

In [ ]:
missing_data = get_missing(df_features)
display(missing_data.head(10))

In [ ]:
numeric_feats = df_features.dtypes[df_features.dtypes != "object"].index

In [ ]:
df_features = pd.get_dummies(df_features)
df_features = df_features.fillna(df_features.mean())
missing_data = get_missing(df_features)
display(missing_data.head(10))

### Target

We're going to check the distribution of the target variable and observe its asymmetry. However, for now, we won't perform any transformations. Then we will fit two models: Lasso and Random Forest.

In [ ]:
print("Skewness: %f" % df_target['SalePrice'].skew())
plot_distribution(df_target, 'SalePrice')

In [ ]:
warnings.simplefilter("ignore", UserWarning)

y = df_target['SalePrice'].astype(float)
X = df_features.reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
dict_models = {
'lasso': {
    'model': Lasso(),
    'space': [
        Real(0, 0.02, name='alpha'),
    ]
},

'rf': {
    'space': [
                Integer(100, 1000, name='n_estimators'),
                Integer(2, 100, name='min_samples_split'),
                Integer(1, 10, name='min_samples_leaf')
                ],
    'model': RandomForestRegressor()}
}

for model in dict_models:
    model_name = model
    space = dict_models[model]['space']
    model = dict_models[model]['model']
    optimizer = Optimizer(space=space, model=model,
                            model_name=model_name, n_calls=20)

    optimizer.find_optimal_params(X=X_train, y=y_train)
    best_model = optimizer.best_model.fit(X_train, y_train.ravel())
    y_pred = best_model.predict(X_test)
    plot_rel = PlotRelations(pd.DataFrame({'y_test': y_test, 'y_pred': y_pred}), f'./pairplot_{model_name}.png')
    plot_rel.plot_graph()
    print(f"Test accuracy -> cor: {pearsonr(y_pred, y_test)[0]:.4f}, mse: {np.mean((y_pred - y_test)**2):.4f}")


#### Correlation map

In [ ]:
vars = list(numeric_feats)
vars.append('SalePrice')
plot_corration_map(pd.concat([df_features, df_target], axis=1)[vars])

Variance inflation factor, VIF, for one exogenous variable

The variance inflation factor is a measure for the increase of the variance of the parameter estimates if an additional variable, is added to the linear regression. It is a measure for multicollinearity of the design matrix.

One recommendation is that if VIF is greater than 5, then the explanatory variable is highly collinear with the other explanatory variables, and the parameter estimates will have large standard errors because of this.

In [ ]:
vif_data = pd.DataFrame()
vif_data["feature"] = numeric_feats
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_features[numeric_feats].values, i)
                          for i in range(len(df_features[numeric_feats].columns))]
  
display(vif_data)

In [ ]:
shapiro_test = stats.shapiro(df_target['SalePrice'])
print(f'H0: The data was drawn from a normal distribution. If pvalue > 0.05, we cannot reject the null hypothesis.')
print(f'Shapiro Test: shapiro.statistic = {shapiro_test.statistic:.4f}, shapiro.pvalue = {shapiro_test.pvalue:.4f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))
sns.boxplot(x=df_target['SalePrice'], ax=ax)
ax.grid(which = "major", axis='both', color='#758D99', zorder=1, linewidth = 0.5, alpha = 0.4,linestyle='-')
ax.grid(which = "minor", axis='both', color='#758D99', zorder=1, linewidth = 0.3, alpha = 0.2,linestyle='-')
ax.minorticks_on()
ax.tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

As we can observe, Random Forest demonstrates superior performance compared to Lasso. Despite our target data initially following a normal distribution, there is multicollinearity among our features. Consequently, employing Ordinary Least Squares (OLS) would not be a suitable option, even though the best Lasso model found has an alpha value of zero. Lasso effectively addresses multicollinearity through regularization.

To enhance the accuracy of Lasso, we will explore a data transformation approach, once the skewness of target variable is equal 1.882876.

In [ ]:
df_target.loc[:,'LogSalePrice'] = np.log1p(df_target['SalePrice'].values)

In [ ]:
plot_distribution(df_target, 'LogSalePrice')
shapiro_test = stats.shapiro(df_target['LogSalePrice'])
print(f'H0: The data was drawn from a normal distribution. If pvalue > 0.05, we cannot reject the null hypothesis.')
print(f'Shapiro Test: shapiro.statistic = {shapiro_test.statistic:.4f}, shapiro.pvalue = {shapiro_test.pvalue:.6f}')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 3))
sns.boxplot(x=df_target['LogSalePrice'], ax=ax)
ax.grid(which = "major", axis='both', color='#758D99', zorder=1, linewidth = 0.5, alpha = 0.4,linestyle='-')
ax.grid(which = "minor", axis='both', color='#758D99', zorder=1, linewidth = 0.3, alpha = 0.2,linestyle='-')
ax.minorticks_on()
ax.tick_params(axis='x', rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
print("Skewness: %f" % df_target['LogSalePrice'].skew())

### Modeling

In [ ]:
warnings.simplefilter("ignore", UserWarning)
y = df_target['LogSalePrice'].astype(float)
X = df_features.reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
dict_models = {
'lasso': {
    'model': Lasso(),
    'space': [
        Real(0, 0.02, name='alpha'),
    ]
},

'rf': {
    'space': [
                Integer(100, 1000, name='n_estimators'),
                Integer(2, 100, name='min_samples_split'),
                Integer(1, 10, name='min_samples_leaf')
                ],
    'model': RandomForestRegressor()}
}

for model in dict_models:
    model_name = model
    space = dict_models[model]['space']
    model = dict_models[model]['model']
    optimizer = Optimizer(space=space, model=model,
                            model_name=model_name, n_calls=20)

    optimizer.find_optimal_params(X=X_train, y=y_train)
    best_model = optimizer.best_model.fit(X_train, y_train.ravel())
    y_pred = best_model.predict(X_test)
    print(f"Test accuracy -> cor: {pearsonr(np.expm1(y_pred), np.expm1(y_test))[0]:.4f}, mse: {np.mean((y_pred - y_test)**2):.4f}")


### Feature engineering

Now let's examine the skewness of our features and apply a transformation to specific variables.

In [ ]:
skewed_feats = df_features[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
display(skewness.sort_values(by=['Skew'], ascending=False).head(10))

In [ ]:
skewed_feats = df_features[numeric_feats].apply(
    lambda x: skew(x.dropna()))  # compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
df_features[skewed_feats] = np.log1p(df_features[skewed_feats])

In [ ]:

skewed_feats = df_features[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
display(skewness.sort_values(by=['Skew'], ascending=False).head(10))

In [ ]:
warnings.simplefilter("ignore", UserWarning)
y = df_target['LogSalePrice'].astype(float)
X = df_features.reset_index(drop=True)

X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
dict_models = {
'lasso': {
    'model': Lasso(),
    'space': [
        Real(0, 0.02, name='alpha'),
    ]
},

'rf': {
    'space': [
                Integer(100, 1000, name='n_estimators'),
                Integer(2, 100, name='min_samples_split'),
                Integer(1, 10, name='min_samples_leaf')
                ],
    'model': RandomForestRegressor()}
}

for model in dict_models:
    model_name = model
    space = dict_models[model]['space']
    model = dict_models[model]['model']
    optimizer = Optimizer(space=space, model=model,
                            model_name=model_name, n_calls=20)

    optimizer.find_optimal_params(X=X_train, y=y_train)
    best_model = optimizer.best_model.fit(X_train, y_train.ravel())
    y_pred = best_model.predict(X_test)
    print(f"Test accuracy -> cor: {pearsonr(np.expm1(y_pred), np.expm1(y_test))[0]:.4f}, mse: {np.mean((y_pred - y_test)**2):.4f}")


Random Forest outperformed Lasso and was minimally affected by data transformations, maintaining its accuracy. On the other hand, Lasso exhibited significantly lower performance when using the data in its original scale, with a correlation coefficient of 0.77, while Random Forest achieved a correlation coefficient of 0.92.

However, after applying transformations to reduce the asymmetry of the target and features, Lasso showed improvement with a correlation coefficient of 0.84, while Random Forest maintained its high accuracy.